<a href="https://colab.research.google.com/github/gwakdh99/gwakdh/blob/master/01_%ED%86%B5%EA%B3%84%EA%B8%B0%EB%B0%98_%EC%9E%90%EC%97%B0%EC%96%B4_%EB%B6%84%EC%84%9D_%EA%B8%B0%EB%B2%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## 형태소 분석 라이브러리 설치 및 호출

### pip, konlpy, nlpy 설치

In [2]:
# pip 업그레이드
!pip install --upgrade pip

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (19.2.3)


In [0]:
# 한글형태소 분석기 konlpy 설치
!pip3 install konlpy

In [0]:
# 형태소분석기 nlpy 설치
!pip3 install nlpy

### 구글드라이브 연동 설정

In [3]:
# 구글드라이브와 연동 설정
from google.colab import drive

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [4]:
 #구글드라이브와 Colab이 잘 연동 되었는지 확인한다.
!ls '/content/drive/'

'My Drive'


### numpy, sklearn 호출

In [0]:
#넘파이 호출
import numpy as np
# 고속 SVD를 위한 sklearn 모듈 호출
from sklearn.utils.extmath import randomized_svd

## 통계기반 분석을 위한 전처리
$ \rm preprocess(text) =$  (corpus,  word_to_id,  id_to_word)





통계 기반 기법을 살펴보기 위해서 특별한 말뭉치(corpus)를 이용할 것입니다. 
펜 트리뱅크(Penn Treebank)(PTB dataset)을 이용하겠습니다. 이 PTB 말뭉치는 텍스트 파일로 제공되며 회소한 단어를 "<unk>"라는 특수문자로, 구체적인 숫자를 "N"으로 치환되어 있습니다.총 용량은 5MB입니다.

In [0]:
# 구글드라이브에 있는 ptb_train.txt 파일을 불러와 read() 메소드로 읽어준다.
f = open("/content/drive/My Drive/Colab Notebooks/ptb_train.txt", 'r')
text = f.read()

In [0]:
# 텍스트 전처리 함수를 설정한다.
def preprocess(text):
    
  # 대용량 텍스트를 공백을 기준으로 분할한다.
    text = text.lower()
    text = text.replace('.',' .')
    words = text.split(' ')
    print(words)
  # word에 id를 부여하자. 
    word_to_id = { }
    id_to_word = { }
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            id_to_word[new_id] = word
            word_to_id[word] = new_id

  # word에 부여된 id를 이용해서 행렬(배열, array)을 생성하자. 
    corpus = [word_to_id[w] for w in words]
    return corpus, word_to_id, id_to_word

In [8]:
corpus, word_to_id, id_to_word = preprocess(text)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
len(corpus)

946313

## 단어의 분산 표현 , 분포가설(Distributional hypothesis)

색은 RGB 컬러로 모두 표현 가능하다. 다시 말해 색은 Red, Green, Blue가 어떤 비율로 섞여 있는지를 3차원 벡터로 표현할 수 있다. <br>
색을 벡터로 표현하듯이 '단어'도 벡터로 표현할 수 있을까? '단어의 의미'를 어떻게 벡터로 표현할 수 있을까? 


> **분포 가설 : 단어의 의미는 주변 단어에 의해 형성된다**



## 동시발생 행렬(Co-occurrence Matrix) 구하기
create_co_matrix(corpus, vocab_size, window_size = 1) = co_matrix

In [0]:
def create_co_matrix(corpus, vocab_size, window_size = 2) : # window_size는 2로 기본 설정되어 있다. 
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype = np.int32 )  # 영행렬을 먼저 생성한다.
    
    for idx, word_id in enumerate(corpus) :     # enumerate 함수는 corpus 리스트의 원소의 순서와 값을 전달하는 기능을 한다. 
        for i in range(1, window_size + 1) :
            left_idx = idx - i
            right_idx = idx + i

            if left_idx >= 0 :
                left_word_id = corpus [left_idx]
                co_matrix[word_id, left_word_id] += 1

            if right_idx < corpus_size : 
                right_word_id = corpus [right_idx]
                co_matrix[word_id, right_word_id] += 1
    return co_matrix

In [11]:
import time
start = time.time()
C = create_co_matrix(corpus, len(id_to_word))
print("코드 실행 시간은 %s초 입니다." %(time.time() - start) )   # 약 8초의 시간이 걸리네요. 

코드 실행 시간은 9.417446613311768초 입니다.


In [24]:
C

array([[0, 1, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

## 코사인 유사도(Cosine Similarity)

cos_similarity(x, y, eps = 1e-8)
</br> 
most_similar(query, word_to_id, id_to_word, word_matrix, top = 5)

벡터의 유사도를 측정하는 방법은 다양합니다. 단어 벡터의 유사도를 나타낼 때는 다음의 코사인 유사도를 자주 이용합니다.
<br/>  
$$ 
\rm cos\_similarity(x, y) = \rm \frac{x\cdot y}{\vert{x}\vert \vert{y}\vert} = \frac {x_1y_1+ \cdots+x_ny_n } {\sqrt{x_1^2+\cdots+x_n^2}\sqrt{y_1^2+\cdots+y_n^2}}
$$

In [0]:
def cos_similarity(x, y, eps = 1e-8) : 

    nx = x / np.sqrt(np.sum(x**2) + eps)
    ny = y / np.sqrt(np.sum(y**2) + eps)
    return np.dot(nx, ny)

### most_similarity를 계산해보자.

In [0]:
def most_similar(query, word_to_id, id_to_word, word_matrix, top = 5):
    # 검색어를 꺼낸다. 
    if query not in word_to_id:
        print('%s 를 찾을 수 없습니다.' % query)
        return
        
    print('[query] : ', query)
    query_id = word_to_id[query]
    query_vec = word_matrix[query_id]
    
    # 코사인 유사도 계산
    vocab_size = len(id_to_word)
    similarity = np.zeros(vocab_size)
    for i in range(vocab_size) : 
        similarity[i] = cos_similarity(word_matrix[i], query_vec)
    
    # 코사인 유사도를 기준으로 내림차순으로 출력
    count = 0
    for i in (-1*similarity).argsort(): 
        if id_to_word[i] == query : 
            continue
        print (' %s : %s ' %(id_to_word[i], similarity[i]))
        
        count += 1 
        if count >= top : 
            return

In [18]:
 cos_similarity(C[word_to_id["the"]], C[word_to_id["car"]])

0.641283895830788

In [19]:
most_similar('korea', word_to_id, id_to_word, C, top = 5)

[query] :  korea
 carolina : 0.9094060136328884 
 africa : 0.8992616581545659 
 gardens : 0.8185984007337492 
 african : 0.7864616290025496 
 korean : 0.7585169775794639 


In [20]:
most_similar('happy', word_to_id, id_to_word, C, top = 5)

[query] :  happy
 sure : 0.6774443817612392 
 need : 0.6632733796166624 
 hope : 0.6617353522854461 
 not : 0.6591013536415403 
 thought : 0.6558402278521558 


In [21]:
most_similar('she', word_to_id, id_to_word, C, top = 5)

[query] :  she
 he : 0.9435928089807651 
 i : 0.860179331917126 
 nobody : 0.8576518710081747 
 but : 0.8572765160046975 
 it : 0.8558084183903918 


## 통계 기반 기법 개선하기, PPMI
ppmi(C, verbose = False, eps = 1e-8)


PMI는 확률변수 x, y에 대하여 다음 식으로 정의됩니다.
</br>  
$$ \rm PMI(\it x, y \rm) = \log_2 \frac{P(\it x,y \rm)}{P(\it x)\rm P(\it y \rm)}$$
</br>  
$ P(x) $는 $x$가 일어날 확률, $ P(x,y) $는 $x, y$가 동시에 일어날 확률을 뜻합니다. 이 PMI 값이 높을수록 관련성이 높다는 의미입니다. 

</br>  
$$ \rm PMI(\it x, y \rm) = \log_2 \frac{P(\it x,y \rm)}{P(\it x)\rm P(\it y \rm)}=\log_2 \frac{\frac{C(x,y)}{N}}{\frac{C(x)}{N}\frac{C(y)}{N}}
=\log_2 \frac{C(x,y)\cdot N}{C(x)C(y)}$$
</br>  

위 식에 따라 동시 발생행렬로부터 PMI를 구할 수 있습니다. 
</br>
예를들어 "the"와 "car"와 "drive"가 각각 1000번, 20번, 10번 등장했다고 해보죠. </br>
그리고 "the"와 "car"의 동시발생 수는 10회, "car"와 "drive"의 동시발생 수는 5회라고 가정합시다. 
</br>
이 조건이라면, 동시발생횟수의 관점에서는 "car"는 "drive"보다 "the"와 관련이 깊다고 나옵니다. 그렇다면 PMI관점에서는 어떨까요? 

In [0]:
def ppmi(C, verbose = False, eps = 1e-8): #verbose = True 이면, 실행과정을 실시간으로 체킹하면서 진행된다.
    M = np.zeros_like(C, dtype = np.float32)
    N = np.sum(C)           # 행렬 C에 있는 모든 성분의 합을 계산한다. 
    S = np.sum(C, axis = 0) # 각 열의 합을 행렬의 열의 성분으로 하는 행렬을 만든다.
    total = C.shape[0] * C.shape[1]
    cnt = 0 

    for i in range(C.shape[0]) : 
        for j in range(C.shape[1]) :
            pmi = np.log2(C[i, j] * 2 * N / (S[j]  * S[i])+ eps)   # 이 수식은 과연 타당한가? window_size가 변해도 불변한가?
            M[i, j] = max(0, pmi)

            # 코드가 작동하는 과정을 알려주는 코드를 작성해봅시다. 
            if verbose : 
                cnt += 1
                if cnt % (total // 100) == 0:
                    print('%.1f%% 완료' % (100*cnt/total)) 
    return M

In [23]:
## 엄청나게 오래 걸리는 군요.....약 462초가 걸립니다. (약 8분 정도)
import time
start = time.time()
W = ppmi(C, True) 
print("time :", time.time() - start)

1.0% 완료
2.0% 완료
3.0% 완료
4.0% 완료
5.0% 완료
6.0% 완료
7.0% 완료
8.0% 완료
9.0% 완료
10.0% 완료
11.0% 완료
12.0% 완료
13.0% 완료
14.0% 완료
15.0% 완료
16.0% 완료
17.0% 완료
18.0% 완료
19.0% 완료
20.0% 완료
21.0% 완료
22.0% 완료
23.0% 완료
24.0% 완료
25.0% 완료
26.0% 완료
27.0% 완료
28.0% 완료
29.0% 완료
30.0% 완료
31.0% 완료
32.0% 완료
33.0% 완료
34.0% 완료
35.0% 완료
36.0% 완료
37.0% 완료
38.0% 완료
39.0% 완료
40.0% 완료
41.0% 완료
42.0% 완료
43.0% 완료
44.0% 완료
45.0% 완료
46.0% 완료
47.0% 완료
48.0% 완료
49.0% 완료
50.0% 완료
51.0% 완료
52.0% 완료
53.0% 완료
54.0% 완료
55.0% 완료
56.0% 완료
57.0% 완료
58.0% 완료
59.0% 완료
60.0% 완료
61.0% 완료
62.0% 완료
63.0% 완료
64.0% 완료
65.0% 완료
66.0% 완료
67.0% 완료
68.0% 완료
69.0% 완료
70.0% 완료
71.0% 완료
72.0% 완료
73.0% 완료
74.0% 완료
75.0% 완료
76.0% 완료
77.0% 완료
78.0% 완료
79.0% 완료
80.0% 완료
81.0% 완료
82.0% 완료
83.0% 완료
84.0% 완료
85.0% 완료
86.0% 완료
87.0% 완료
88.0% 완료
89.0% 완료
90.0% 완료
91.0% 완료
92.0% 완료
93.0% 완료
94.0% 완료
95.0% 완료
96.0% 완료
97.0% 완료
98.0% 완료
99.0% 완료
100.0% 완료
time : 463.1269793510437


In [25]:
most_similar('korea', word_to_id, id_to_word, W, top = 5)
print()
print("비교해 봅시다. 다음은 동시발생 행렬에 의한 유사도 순위입니다.")
print(most_similar('korea', word_to_id, id_to_word, C, top = 5))

[query] :  korea
 south : 0.18941046297550201 
 panels : 0.1284870207309723 
 korean : 0.1282111257314682 
 kong : 0.12692923843860626 
 provinces : 0.12193956971168518 

비교해 봅시다. 다음은 동시발생 행렬에 의한 유사도 순위입니다.
[query] :  korea
 carolina : 0.9094060136328884 
 africa : 0.8992616581545659 
 gardens : 0.8185984007337492 
 african : 0.7864616290025496 
 korean : 0.7585169775794639 
None


In [26]:
most_similar('happy', word_to_id, id_to_word, W, top = 5)
print()
print("비교해 봅시다. 다음은 동시발생 행렬에 의한 유사도 순위입니다.")
print(most_similar('happy', word_to_id, id_to_word, C, top = 5))

[query] :  happy
 ok : 0.19403694570064545 
 pleased : 0.18708181381225586 
 lucky : 0.18692053854465485 
 surprised : 0.1794547438621521 
 glad : 0.17548418045043945 

비교해 봅시다. 다음은 동시발생 행렬에 의한 유사도 순위입니다.
[query] :  happy
 sure : 0.6774443817612392 
 need : 0.6632733796166624 
 hope : 0.6617353522854461 
 not : 0.6591013536415403 
 thought : 0.6558402278521558 
None


In [28]:
most_similar('car', word_to_id, id_to_word, W, top = 5)
print()
print("비교해 봅시다. 다음은 동시발생 행렬에 의한 유사도 순위입니다.")
print(most_similar('car', word_to_id, id_to_word, C, top = 5))

[query] :  car
 auto : 0.18711645901203156 
 cars : 0.16278202831745148 
 rental : 0.13010603189468384 
 domestic : 0.11845936626195908 
 chevrolet : 0.11274931579828262 

비교해 봅시다. 다음은 동시발생 행렬에 의한 유사도 순위입니다.
[query] :  car
 network : 0.8740191057446254 
 job : 0.8724513724339986 
 strategy : 0.8661775022309377 
 group : 0.8632326434651192 
 computer : 0.8620327743208449 
None


Co_occurence matrix를 PPMI matrix로 변환했는데도 크게 향상된 것 같지는 않군요. 사실 PPMI matrix에도 몇가지 보완해야 할 점이 있습니다.
 행렬에 0을 너무 많이 포함하고 있습니다. 이는 코사인유사도로 검증을 할 때 좋지 못한 점입니다. 이렇게 원소의 대부분이 0인 행렬을 '희소행렬(Sparse matrix)'라고 합니다. </br> 
'희소행렬(Sparse matrix)'을 원소의 대부분이 0이 아닌 '밀집벡터'로 표현하는 것이 우리의 최종 목표입니다.

## 고속 SVD를 이용해서 PPMI의 차원을 축소하기
U, Sigma, VT = randomized_svd(행렬, 차원)

앞의 과정에서 PPMI 행렬로도 큰 개선의 효과를 보지 못했습니다. 이는 PPMI 행렬의 대부분의 성분이 0 이기 때문에 발생하는 입니다. 여기서는 SVD에 의하여 PPMI 행렬을 대부분의 성분이 0이 없는 행렬로 만들면서 행렬의 차원을 축소하는 방법을 시도하려고 합니다. 

단어의 의미를 포함하고 있는 행렬은 U이기 때문에 이 행렬을 단어벡터의 행렬로 두기로 합니다.

In [48]:
import time
start = time.time()
U, S, VT = randomized_svd(W, n_components = 100, n_iter = 5, random_state = None)
print( "time :", time.time() - start)

time : 6.794035911560059


In [0]:
US = np.dot(U, np.diag(S))

In [51]:
print("U 행렬의 형태 : " , U.shape)
print(np.round(U, 10))
print()
print("S 행렬의 형태 : " , S.shape)
print(np.round(np.diag(Sigma), 3))
print()
print("VT 행렬의 형태 : " , VT.shape)
print(np.round(VT, 3))

print()
print("PPMI 행렬인 W 행렬의 형태 : " , W.shape)
print(np.round(W, 3))

print()
print("U*S 행렬의 형태 : " , np.dot(U, np.diag(S)).shape)
print(np.round(np.dot(U, np.diag(S)), 3))

print()
print("W*V 행렬의 형태 : " , np.dot(W, VT.T).shape)
print(np.round(np.dot(W, np.dot(W, VT.T)), 3))

U 행렬의 형태 :  (9945, 100)
[[ 1.0000000e-10 -5.7789998e-07  5.5639998e-07 ... -2.4390600e-05
  -4.7103999e-06  3.4349399e-05]
 [ 0.0000000e+00  2.2000000e-09 -3.0000000e-10 ... -3.2472100e-05
  -1.5593600e-05  4.3846401e-05]
 [ 0.0000000e+00 -4.0000001e-10 -2.7000000e-09 ... -4.4023400e-05
  -1.9863999e-05  6.6485998e-05]
 ...
 [ 4.1807014e-03 -6.4211129e-03  8.0438042e-03 ...  3.7873115e-03
  -7.4778832e-03  3.5672812e-03]
 [ 2.2456090e-03 -4.4795838e-03  9.0071950e-03 ...  2.6698033e-03
  -1.2235908e-03  2.8866279e-04]
 [ 3.4797485e-03 -6.5582674e-03  1.0606006e-02 ...  2.9114669e-03
  -6.6688708e-03 -3.3550407e-03]]

S 행렬의 형태 :  (100,)
[[956.464   0.      0.    ...   0.      0.      0.   ]
 [  0.    463.054   0.    ...   0.      0.      0.   ]
 [  0.      0.    336.267 ...   0.      0.      0.   ]
 ...
 [  0.      0.      0.    ... 126.706   0.      0.   ]
 [  0.      0.      0.    ...   0.    126.601   0.   ]
 [  0.      0.      0.    ...   0.      0.    126.22 ]]

VT 행렬의 형태 :  (100, 

In [58]:
print(" 다음은 U 행렬로 계산한 유사도입니다. ")
print('the , drive 사이의 유사도는 ')
print(cos_similarity(U[word_to_id['the']], U[word_to_id['car']]))
print()
print("car , drive 사이의 유사도는 ")
print(cos_similarity(U[word_to_id['car']], U[word_to_id['drive']]))
print()
print()
print(" 다음은 US 행렬로 계산한 유사도입니다. ")
print('the , drive 사이의 유사도는 ')
print(cos_similarity(US[word_to_id['the']], US[word_to_id['car']]))
print()
print("car , drive 사이의 유사도는 ")
print(cos_similarity(US[word_to_id['car']], US[word_to_id['drive']]))

 다음은 U 행렬로 계산한 유사도입니다. 
the , drive 사이의 유사도는 
-0.034210715

car , drive 사이의 유사도는 
0.32783842


 다음은 US 행렬로 계산한 유사도입니다. 
the , drive 사이의 유사도는 
0.33936846

car , drive 사이의 유사도는 
0.5631735


In [63]:
print("다음은 U 행렬로 계산한 유사도입니다. ")
print('a , man 사이의 유사도는 ')
print(cos_similarity(U[word_to_id['a']], U[word_to_id['man']]))
print()
print("man , woman 사이의 유사도는 ")
print(cos_similarity(U[word_to_id['man']], U[word_to_id['woman']]))
print()
print()
print("다음은 US 행렬로 계산한 유사도입니다. ")
print('a , man 사이의 유사도는 ')
print(cos_similarity(US[word_to_id['a']], US[word_to_id['man']]))
print()
print("man , woman 사이의 유사도는 ")
print(cos_similarity(US[word_to_id['man']], US[word_to_id['woman']]))

다음은 U 행렬로 계산한 유사도입니다. 
a , man 사이의 유사도는 
0.074401334

man , woman 사이의 유사도는 
0.43411198


다음은 US 행렬로 계산한 유사도입니다. 
a , man 사이의 유사도는 
0.36367357

man , woman 사이의 유사도는 
0.69762295


In [54]:
most_similar('korea', word_to_id, id_to_word, US, top = 5)
print()
print()
print()
most_similar('korea', word_to_id, id_to_word, U, top = 5)
print()
print("비교해 봅시다. 다음은 PPMI 행렬에 의한 유사도 순위입니다.")
most_similar('korea', word_to_id, id_to_word, W, top = 5)
print()
print("비교해 봅시다. 다음은 동시발생 행렬에 의한 유사도 순위입니다.")
print(most_similar('korea', word_to_id, id_to_word, C, top = 5))

[query] :  korea
 taiwan : 0.7533437013626099 
 europe : 0.7332642078399658 
 south : 0.7153602838516235 
 india : 0.6929842829704285 
 china : 0.6923523545265198 



[query] :  korea
 taiwan : 0.635428249835968 
 south : 0.6304270029067993 
 europe : 0.6040027141571045 
 japan : 0.5510867834091187 
 canada : 0.5441176295280457 

비교해 봅시다. 다음은 PPMI 행렬에 의한 유사도 순위입니다.
[query] :  korea
 south : 0.18941046297550201 
 panels : 0.1284870207309723 
 korean : 0.1282111257314682 
 kong : 0.12692923843860626 
 provinces : 0.12193956971168518 

비교해 봅시다. 다음은 동시발생 행렬에 의한 유사도 순위입니다.
[query] :  korea
 carolina : 0.9094060136328884 
 africa : 0.8992616581545659 
 gardens : 0.8185984007337492 
 african : 0.7864616290025496 
 korean : 0.7585169775794639 
None


In [55]:
most_similar('car', word_to_id, id_to_word, US, top = 5)
print()
print()
print()

most_similar('car', word_to_id, id_to_word, U, top = 5)
print()
print("비교해 봅시다. 다음은 PPMI 행렬에 의한 유사도 순위입니다.")
most_similar('car', word_to_id, id_to_word, W, top = 5)
print()
print("비교해 봅시다. 다음은 동시발생 행렬에 의한 유사도 순위입니다.")
print(most_similar('car', word_to_id, id_to_word, C, top = 5))

[query] :  car
 auto : 0.7355228066444397 
 luxury : 0.6983713507652283 
 cars : 0.6808823347091675 
 truck : 0.6740310788154602 
 domestic : 0.6408664584159851 



[query] :  car
 auto : 0.5921629667282104 
 luxury : 0.5740488767623901 
 cars : 0.5352737307548523 
 truck : 0.5105409622192383 
 lexus : 0.4421027898788452 

비교해 봅시다. 다음은 PPMI 행렬에 의한 유사도 순위입니다.
[query] :  car
 auto : 0.18711645901203156 
 cars : 0.16278202831745148 
 rental : 0.13010603189468384 
 domestic : 0.11845936626195908 
 chevrolet : 0.11274931579828262 

비교해 봅시다. 다음은 동시발생 행렬에 의한 유사도 순위입니다.
[query] :  car
 network : 0.8740191057446254 
 job : 0.8724513724339986 
 strategy : 0.8661775022309377 
 group : 0.8632326434651192 
 computer : 0.8620327743208449 
None


In [56]:
most_similar('happy', word_to_id, id_to_word, US, top = 5)
print()
print()
print()
most_similar('happy', word_to_id, id_to_word, U, top = 5)
print()
print("비교해 봅시다. 다음은 PPMI 행렬에 의한 유사도 순위입니다.")
most_similar('happy', word_to_id, id_to_word, W, top = 5)
print()
print("비교해 봅시다. 다음은 동시발생 행렬에 의한 유사도 순위입니다.")
print(most_similar('happy', word_to_id, id_to_word, C, top = 5))

[query] :  happy
 interested : 0.765058696269989 
 confident : 0.7429779767990112 
 aware : 0.7428470849990845 
 feeling : 0.7373619675636292 
 afraid : 0.7369123697280884 



[query] :  happy
 feeling : 0.5601326823234558 
 interested : 0.5579146146774292 
 afraid : 0.5442239046096802 
 ready : 0.4918520450592041 
 convinced : 0.49076756834983826 

비교해 봅시다. 다음은 PPMI 행렬에 의한 유사도 순위입니다.
[query] :  happy
 ok : 0.19403694570064545 
 pleased : 0.18708181381225586 
 lucky : 0.18692053854465485 
 surprised : 0.1794547438621521 
 glad : 0.17548418045043945 

비교해 봅시다. 다음은 동시발생 행렬에 의한 유사도 순위입니다.
[query] :  happy
 sure : 0.6774443817612392 
 need : 0.6632733796166624 
 hope : 0.6617353522854461 
 not : 0.6591013536415403 
 thought : 0.6558402278521558 
None


In [57]:
most_similar('intel', word_to_id, id_to_word, US, top = 5)
print()
print()
print()

most_similar('intel', word_to_id, id_to_word, U, top = 5)
print()
print("비교해 봅시다. 다음은 PPMI 행렬에 의한 유사도 순위입니다.")
most_similar('intel', word_to_id, id_to_word, W, top = 5)
print()
print("비교해 봅시다. 다음은 동시발생 행렬에 의한 유사도 순위입니다.")
print(most_similar('intel', word_to_id, id_to_word, C, top = 5))

[query] :  intel
 hewlett-packard : 0.7207717895507812 
 dell : 0.7184058427810669 
 nec : 0.701552152633667 
 sun : 0.6988478302955627 
 compaq : 0.6938881278038025 



[query] :  intel
 lsi : 0.5289038419723511 
 hewlett-packard : 0.5281217098236084 
 armonk : 0.5276092886924744 
 dell : 0.5177016854286194 
 nec : 0.5142529010772705 

비교해 봅시다. 다음은 PPMI 행렬에 의한 유사도 순위입니다.
[query] :  intel
 tandy : 0.18660297989845276 
 ncr : 0.15885891020298004 
 hewlett-packard : 0.1451587975025177 
 shell : 0.14351068437099457 
 convex : 0.14042752981185913 

비교해 봅시다. 다음은 동시발생 행렬에 의한 유사도 순위입니다.
[query] :  intel
 exxon : 0.8507228218177914 
 amoco : 0.8410354286977014 
 chrysler : 0.8382270578539716 
 bankamerica : 0.8329715314930315 
 microsoft : 0.8179202900654178 
None


## 정리하기

지금까지 했던 과정을 다시 되돌려 보면 다음과 같은 순서로 진행되었다. 
1. 텍스트 파일의 전처리 : corpus, word_to_id, id_to_word 를 생성
2. 동시발생 행렬의 생성 : 코사인유사도를 이용하여 co_matrix 를 생성
3. PPMI 행렬의 생성 : ppmi 를 생성 
4. PPMI 행렬을 SVD를 통해서 밀집벡터 표현을 생성 : truncated SVD 를 이용하여  word_vecs 을 생성